In [1]:
import pandas as pd
import numpy as np

In [2]:
def energy_part():
    energy =  pd.read_excel('Energy Indicators.xls', nrows = 227, skiprows=17, usecols = 'C:F', names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])
    
    energy.loc[energy['Energy Supply']!=('...'), 'Energy Supply'] = energy['Energy Supply'] * (10**6)
    energy.loc[energy['Energy Supply']==('...'), 'Energy Supply'] = np.NaN
    energy.loc[energy['Energy Supply per Capita']==('...'), 'Energy Supply per Capita'] = np.NaN  
    
    change = [
    ['Republic of Korea', 'United States of America', 'United Kingdom of Great Britain and Northern Ireland', 'China, Hong Kong Special Administrative Region'],
    ['South Korea', 'United States', 'United Kingdom', 'Hong Kong']
    ]
    
    for i in range(len(change[0])):
        energy.loc[energy['Country'].str.startswith(change[0][i]), 'Country'] = change[1][i]
    
    banned = '\(|0|1|2|3|4|5|6|7|8|9'
    right_version = list()
    need_to_change = energy.loc[energy['Country'].str.contains(banned), 'Country'].to_numpy()
    temp = energy.loc[energy['Country'].str.contains(banned), 'Country'].to_numpy()
    
    for i in temp:
        if(i.__contains__('(') != -1):
            i = ((i[:i.find('(')]+i[i.find(')'):]).replace(')', '')) 
        if(i.__contains__(banned)!=-1):
            for c in banned.split('|'):
                i=i.replace(c, '')
        i = i.strip()
        right_version.append(i)
        
    for i in range(len(need_to_change)):
        energy.loc[energy['Country'] == need_to_change[i], 'Country'] = right_version[i]

    return energy

In [3]:
def GDP_part():
    GDP =  pd.read_excel('word_bank.xls', skiprows=3)

    change = [
        ['Korea, Rep.', 'Iran, Islamic Rep.', 'Hong Kong SAR, China'],
        ['South Korea', 'Iran', 'Hong Kong']
        ]
    
    for i in range(len(change[0])):
        GDP.loc[GDP['Country Name'].str.startswith(change[0][i]), 'Country Name'] = change[1][i]
    GDP.rename(columns = {'Country Name' : 'Country'}, inplace = True)
    
    return GDP

In [4]:
def scimEn_part():
    ScimEn =  pd.read_excel('scimagojr country rank 1996-2022.xlsx')
    ScimEn = ScimEn.loc[ScimEn['Rank']<16]
    
    return ScimEn

In [5]:
def answer_one():
    energy = energy_part()
    GDP = GDP_part()
    ScimEn = scimEn_part()
    
    scimEnColumns = ['Rank', 'Country', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index']
    GDPColumns = ['Country', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    energyColumns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    merged = ScimEn[scimEnColumns].merge(energy[energyColumns], how = 'inner', left_on = 'Country', right_on = 'Country')
    merged = merged.merge(GDP[GDPColumns], how = 'inner', left_on = 'Country', right_on = 'Country')
    merged = merged.set_index('Country')
    
    return merged

In [6]:
def answer_two():
    Top15 = answer_one()
    d = (((Top15[Top15.columns[10:20]]).sum(axis = 1))/10.0)
    
    Top15_AVG = pd.Series(data = d, name = 'avgGDP')
    Top15_AVG = Top15_AVG.sort_values(ascending = False)
    
    return Top15_AVG

In [7]:
def answer_three():
    Top15 = answer_one()
    Top15_AVG = pd.DataFrame(answer_two())
    Top_6th_country = Top15_AVG.iloc[5].name
    difference = Top15.loc[Top_6th_country][19] - Top15.loc[Top_6th_country][10] 
    return difference

In [8]:
def answer_four():
    Top15 = answer_one()
    result = pd.DataFrame(Top15['Self-citations'] / Top15['Citations'], columns = ['Ratio'])
    biggest = result.idxmax()
    return tuple(zip(biggest, result.loc[biggest]['Ratio']))

In [9]:
def answer_five():
    Top15 = answer_one()
    result = pd.DataFrame(Top15['Energy Supply']/Top15['Energy Supply per Capita'], columns = ['Population'])
    result = result.sort_values(by = 'Population', ascending = False)
    return result.iloc[2].name

In [10]:
def answer_six():
    Top15 = answer_one()
    population = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    citable_documents_per_Capita = pd.DataFrame(Top15['Citable documents']/population, columns = ['Citable Documents per Capita'])
    dfs = [citable_documents_per_Capita, Top15['Energy Supply per Capita']]
    result = pd.DataFrame(pd.concat(dfs, axis = 1), columns = ['Citable Documents per Capita','Energy Supply per Capita'])
    return (result.corr(method = 'pearson')).loc['Citable Documents per Capita']['Energy Supply per Capita']

In [11]:
def answer_seven():
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    
    Top15 = answer_one()
    
    population = pd.DataFrame(Top15['Energy Supply']/Top15['Energy Supply per Capita'], columns = ['Population'])
    
    continents_val = np.array(list(ContinentDict.values()))
    countries_val = np.array(list(ContinentDict.keys()))
    continents_unique = {key: list() for key in np.unique(continents_val)}
    
    for i in range(countries_val.size):
        continents_unique[continents_val[i]].append(countries_val[i])
    
    Continent = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'],index = [list(continents_unique.keys())])
    new_values = list()
    
    for i in list(continents_unique):
        new_values.append(len(continents_unique[i]))
    
    Continent['size'] = new_values
    
    new_values.clear()
    
    for i in list(continents_unique):
        sum = 0.0
        for j in continents_unique[i]:
            sum += (population.loc[j][0])
        new_values.append(sum)
        sum = 0.0
        
    Continent['sum'] = new_values
    Continent['mean'] = Continent['sum']/Continent['size']
    Continent['std'] = Continent['mean']**(1/2)
    
    return Continent

In [12]:
print('Answer one:')
print(answer_one(), end ='\n'*2)
print('Answer two:')
print(answer_two(), end = '\n'*2)
print('Answer three:')
print(answer_three(), end = '\n'*2)
print('Answer four:')
print(answer_four(), end ='\n'*2)
print('Answer five:')
print(answer_five(), end = '\n'*2)
print('Answer six:')
print(answer_six(), end = '\n'*2)
print('Answer seven:')
print(answer_seven(), end = '\n'*2)

Answer one:
                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     360468             358777    3947871   
United States          2     199442             195042    3068926   
India                  3      76103              74167     760964   
Japan                  4      56249              55680     633294   
United Kingdom         5      52572              51156     909276   
Germany                6      47781              46767     641717   
Russian Federation     7      43567              43290     175721   
Canada                 8      39036              38276     787010   
Italy                  9      35991              34424     529459   
South Korea           10      35294              35005     503147   
Iran                  11      29776              29448     511199   
France                12      29351              28759     471469   
Spain                 

C:\Users\vital\AppData\Local\Temp\ipykernel_7128\1441937621.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = Top15.loc[Top_6th_country][19] - Top15.loc[Top_6th_country][10]


118652421857.7959

Answer four:
(('China', 0.6853754846599598),)

Answer five:
United States

Answer six:
0.7114342527331206

Answer seven:
               size           sum          mean           std
Asia              5  2.898666e+09  5.797333e+08  24077.650993
Australia         1  2.331602e+07  2.331602e+07   4828.666205
Europe            6  4.579297e+08  7.632161e+07   8736.224082
North America     2  3.528552e+08  1.764276e+08  13282.606097
South America     1  2.059153e+08  2.059153e+08  14349.747532



C:\Users\vital\AppData\Local\Temp\ipykernel_7128\620512434.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sum += (population.loc[j][0])
